Copyright (c) Microsoft Corporation.
Licensed under the MIT license.

## Model Scoring
Load data and generate forecast with trained model:

1. Define variables
2. Connect to AML
3. Retrieve Model and Data
4. Score on data
5. Store results

In [ ]:
from azureml.core import  Workspace, Model
import pickle
from pyspark.sql import functions as F

## Define variables


In [ ]:
# Define Dataset name and Azure Storage path
subscription_id = ""
resource_group = ""
workspace_name = ""
model_name = "commodity-price-forecast-Best"
table_input = "default.test"
data_lake_account_name = ""
file_system_name = ""
scored_data_path = "abfss://%s@%s.dfs.core.windows.net/Result/Scored" % (file_system_name, data_lake_account_name)


## Connect to AML


In [ ]:
# Connect to Azure ML workspace
ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)

## Retrieve Model and Data


In [ ]:
# Retrieve trained model
model_path = Model(ws,model_name).download(exist_ok=True)

# Load model into notebook
with open(model_path+'/model.pkl','rb') as mp:
    model = pickle.load(mp)

In [ ]:
# Get holdout testset
df = spark.sql(f"SELECT * FROM {table_input}")
df = df.withColumn('Date', F.to_timestamp(F.col('Date'),'yyyy-MM-dd')).toPandas()
df = df.sort_values("Date")
#display(df)

## Score model on Data


In [ ]:
# Generate 'forecast'
df['predicted_value'] = model.forecast(df)[0]

## Store Results


In [ ]:
# Store results
df_spark = spark.createDataFrame(df)
df_spark.write.mode('overwrite').save(scored_data_path,format='parquet')
df_spark.write.mode("overwrite").saveAsTable('scored') 